<a href="https://colab.research.google.com/github/hndrr/colab/blob/main/facedetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install insightface
!pip install onnxruntime
!pip install opencv-python-headless

In [ ]:
import gradio as gr
import cv2
import numpy as np
import tempfile
from PIL import Image
from zipfile import ZipFile
import insightface
from insightface.app import FaceAnalysis


In [ ]:
# InsightFaceの初期化
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))


In [ ]:
def make_square_bbox(bbox):
    x1, y1, x2, y2 = bbox
    w = x2 - x1
    h = y2 - y1
    side_len = max(w, h)
    center_x, center_y = x1 + w // 2, y1 + h // 2
    new_x1 = max(center_x - side_len // 2, 0)
    new_y1 = max(center_y - side_len // 2, 0)
    new_x2 = new_x1 + side_len
    new_y2 = new_y1 + side_len
    return [new_x1, new_y1, new_x2, new_y2]

def resize_image(img, target_size):
    return cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)

def detect_and_crop_faces(file_list):
    with tempfile.TemporaryDirectory() as tmp_dir:
        face_count = 0
        for file_path in file_list:
            img = cv2.imread(file_path)
            faces = app.get(img)

            for face in faces:
                bbox = make_square_bbox(face['bbox'])
                x1, y1, x2, y2 = map(int, bbox)
                face_img = img[y1:y2, x1:x2]
                face_img = resize_image(face_img, 500)
                pil_face = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
                pil_face.save(f'{tmp_dir}/face_{face_count}.png')
                face_count += 1

        if face_count == 0:
            return "顔が検出されませんでした。", None

        zip_buffer = tempfile.NamedTemporaryFile(delete=False)
        with ZipFile(zip_buffer.name, 'w') as zipf:
            for i in range(face_count):
                zipf.write(f'{tmp_dir}/face_{i}.png', f'face_{i}.png')

        return "顔検出完了。ZIPファイルをダウンロードしてください。", zip_buffer.name

def clear_inputs():
    return [], "入力をクリアしました。", None  # ファイル入力を空のリストでリセット


In [ ]:
with gr.Blocks() as facedetector:
    with gr.Row():
        with gr.Column():
            file_input = gr.File(file_types=["image"], file_count="multiple", label="Upload Image Files")
            clear_button = gr.Button("クリア")
            submit_button = gr.Button("顔検出", variant="primary")
        with gr.Column():
            output_label = gr.Label(value="結果表示")
            download_link = gr.File(label="Download Zip")

    file_input.change(detect_and_crop_faces, inputs=[file_input], outputs=[output_label, download_link])
    submit_button.click(detect_and_crop_faces, inputs=[file_input], outputs=[output_label, download_link])
    clear_button.click(clear_inputs, outputs=[file_input, output_label, download_link])

facedetector.launch()
